# Data Manipulation Notebook
by Arun Suresh, Nick Wheatley and James Conner

<hr style="border:2px solid gray">

##### This notebook will showcase the various data manipulation techniques used to clean and transform our main datasets (Zillow, ACS, Equifax) into working datasets for use.

<hr style="border:1px solid gray">

## Additional Notebooks

**[Data Merging and Index Creation](02_Data_Merging_and_Index_Creation.ipynb)**

**[Data Analysis](03_Analysis.ipynb)**

<hr style="border:1px solid gray">

## Notebook Setup

### Set Up the Environment

Ensure that the environment has consistent versions of the required libraries.

In [1]:
import sys
# Pandas required for dataframes
# Numpy required for pandas
# Pyarrow installed to write/read parquet files
# Pip upgrade required due to pandas wheel build issues

required_libs = ['pandas==1.2.5', 'numpy==1.23.4', 'pyarrow==9.0.0']
required_libs_str = " ".join(required_libs)

current_libs = !{sys.executable} -m pip freeze
                    
if len(set(required_libs) - set(current_libs)) != 0:
    print("Missing or different libraries.")
    print("Installing required libraries/versions.")
    !{sys.executable} -m pip install --upgrade pip -q 
    !{sys.executable} -m pip install -q {required_libs_str}
else:
    print("Libraries and versions match.")

Missing or different libraries.
Installing required libraries/versions.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yfinance 0.2.10 requires pandas>=1.3.0, but you have pandas 1.2.5 which is incompatible.
scipy 1.6.2 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.23.4 which is incompatible.
fastparquet 2023.1.0 requires pandas>=1.5.0, but you have pandas 1.2.5 which is incompatible.


### Import Libraries/Modules

In [2]:
import os
import pandas as pd
import numpy as np
from collections import defaultdict

<hr style="border:1px solid gray">

### Data Manipulation

#### Zillow

Let's first start with our Zillow data. For our analysis, we used Zillow's ZHVI (Zillow Home Value Index) dataset, which provides typical home values per county going back all the way to 2000. Median Home Values composed a large part (weight) of our Affordability Index, a metric we created as a means to measure housing affordability on a per-county basis. We'll get into further detail about the indices used in our analysis in a separate notebook. 

##### Read files

Each dataset was downloaded as a CSV to start. Here we also download a file containing full US state names to replace Zillow's abbreviated State names.

In [3]:
# read in data files
zillow = pd.read_csv('data/Zillow_All_Homes_TimeSeries_Smoothed_Seasonally_Adjusted_By_County.csv')
state_names = pd.read_parquet('data/state_names.parquet')

Let's inspect the head to see how the data is constructed.

In [4]:
zillow.head(2)

,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,2000-01-31,...,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30
0,3101,0,Los Angeles County,county,CA,CA,"Los Angeles-Long Beach-Anaheim, CA",6,37,216545.0,...,832178.0,846832.0,864584.0,879885.0,881568.0,881064.0,867874.0,857204.0,845637.0,843637.0
1,139,1,Cook County,county,IL,IL,"Chicago-Naperville-Elgin, IL-IN-WI",17,31,174229.0,...,299175.0,301732.0,303877.0,307321.0,310111.0,312096.0,312582.0,313144.0,313615.0,314365.0


##### Transforming the Data

As you can see, our data is constructed in a wide-format, with each month-year combination being its own column, not very informative for our purposes. This makes it hard to group and aggregate and ultimately merge our data on State, County, and Year so we'll have to melt our data from wide to long. 

In addition, part of our analysis and visualizations require the use of FIPS codes to identify different counties. Because some FIPS codes have pre-pended 0's, here we pad the State FIPS and Municipal FIPS with 0's as CSV files tend to truncate them upon reading. Let's also drop non-informative columns such as *SizeRank, RegionType, and RegionID* and only work with years 2016 and beyond. 

In [5]:
# rename 'RegionName' column to 'County' for merging purposes 
zillow = zillow.rename(columns={'RegionName': 'County'})

# limit years to 2016 onwards to focus on 2017-2022 timeframe
years = \
[str(year) for year in range(2000, 2016)]

cols_to_drop = \
[col for col in zillow.columns for year in years if year in col]


# do not drop existing zillow fips columns, 
# but make a combine/padded column with state fips and municipal fips, 
# add municipal fips to state fips
# add zfill to the str to help create FIPS codes

zillow['Fips'] = \
zillow['StateCodeFIPS'].astype('str').str.zfill(2) + zillow['MunicipalCodeFIPS'].astype('str').str.zfill(3)

# drop non-informative columns not used for analysis
cols_to_drop\
.extend(['SizeRank', 'RegionType', 'StateName', 'RegionID', 'Metro'])
zillow = zillow.drop(columns=cols_to_drop)

# convert zillow df from wide to long for easy merging & comparison of feature columns
zillow = \
zillow.melt(id_vars=['County', 'State', 'StateCodeFIPS', 'MunicipalCodeFIPS', 'Fips'], var_name='Year', \
value_name='Home Value')

zillow['Year'] = zillow['Year'].apply(lambda x: x[:4])
zillow.Year = zillow.Year.astype('int')

##### Split-Apply-Combine

Our data came in month-year combinations, e.g. 11/2021, but we only want one datapoint (median home value) per year since we are looking at yearly variations to housing affordability around the pandemic years, so let's group our Zillow columns and take the Median Home Value. Finally we convert the grouped series to a dataframe. 

In [6]:
# group zillow df by all columns except for Home Value in order 
# to take median home value by county/state/year/fips codes
# and convert to new dataframe

zillow = \
zillow.groupby(['County', 'State', 'Year', 'StateCodeFIPS', \
'MunicipalCodeFIPS', 'Fips'])\
['Home Value'].median().to_frame('Median Home Value')

# reset index to re-align columns
zillow = zillow.reset_index()

Inspecting the head again, now we see each County, State, and Year combination has a unique Median Home Value associated with it.

In [7]:
zillow.head(7)

,County,State,Year,StateCodeFIPS,MunicipalCodeFIPS,Fips,Median Home Value
0,Abbeville County,SC,2016,45,1,45001,99338.5
1,Abbeville County,SC,2017,45,1,45001,102073.5
2,Abbeville County,SC,2018,45,1,45001,114003.0
3,Abbeville County,SC,2019,45,1,45001,120961.5
4,Abbeville County,SC,2020,45,1,45001,126615.0
5,Abbeville County,SC,2021,45,1,45001,143518.0
6,Abbeville County,SC,2022,45,1,45001,165914.0


##### String Manipulation

Because the other main datasets, Equifax and ACS, contain full state names, let's replace the abbreviated state names with the full ones for Zillow (to ease with merging on State) along with other string manipulations to strip white space and fix county names. 

In [8]:
# use state_names df to replace abbreviated zillow state names to full state names
state_names['state'] = state_names['state'].str.strip()
state_names['code'] = state_names['code'].str.strip()

zillow['State'] = \
zillow['State'].replace(list(state_names['code']), \
list(state_names['state']))

zillow['State'] = zillow['State'].str.strip()

# Apply string manipulations to fix county names and get rid of the word 'County'.
# Louisiana and Alaska don't use 'County' to delineate municipalities but rather
# 'Parish' and 'Borough' respectively. So let's get rid of 'Parish' and 'Borough'. 
zillow['County'] = \
zillow['County'].str.replace("County", "", regex=False)

zillow['County'] = \
zillow['County'].mask(zillow['State'] == 'Alaska', \
zillow['County'].str.replace('Borough', ''))

zillow['County'] = \
zillow['County'].mask(zillow['State'] == 'Louisiana', \
zillow['County'].str.replace('Parish', ''))

zillow['County'] = zillow['County'].str.strip()

# Further inspection of county names resulted in misspelled/inconsistent County names.
# Let's fix that here.
zillow.County = zillow.County.str.replace("De Kalb", "DeKalb")
zillow.County = zillow.County.str.replace("Dekalb", "DeKalb")
zillow.County = zillow.County.str.replace("La Salle", "LaSalle")
zillow.County = zillow.County.str.replace("Dewitt", "DeWitt")
zillow.County = zillow.County.str.replace("De Soto", "DeSoto")
zillow.County = zillow.County.str.replace("O Brien", "O'Brien")

Now, let's check for any nan values and any major outliers in the data.

In [9]:
# nan ratio is rather small, so dropping is fine here
zillow.groupby(['Year'])['Median Home Value'].apply(lambda x: x.isna().sum() / x.count())

Year
2016    0.096413
2017    0.077302
2018    0.057270
2019    0.034947
2020    0.018631
2021    0.010306
2022    0.000000
Name: Median Home Value, dtype: float64

In [10]:
zillow = zillow.dropna()
zillow.loc[:, 'Median Home Value'] = zillow['Median Home Value'].apply(lambda x: round(x, 2))

In [11]:
zillow['Median Home Value'].describe()

count    1.911600e+04
mean     1.892444e+05
std      1.321232e+05
min      3.137600e+04
25%      1.125035e+05
50%      1.543960e+05
75%      2.236511e+05
max      2.317702e+06
Name: Median Home Value, dtype: float64

In [12]:
# write file to parquet to maintain the dtypes and for faster read/write
zillow.to_parquet('data/zillow.parquet')

#### Equifax

Now, onto the Equifax data. The Equifax dataset used provides county level shelter and utilities data for the year 2022 and their respective projections for the year 2027. Utilities such as gas and electricity are provided, but for the purposes of our analysis, we will take the sum of all utilities on a county/state/year level. Total utilties form another component of our main Affordability Index metric used to measure housing affordability. 

##### Read files

Here we're reading the Zillow data as well since we're using the Zillow counties as the base counties for our analysis, since Zillow contains the home values.

In [13]:
zillow = pd.read_parquet('data/zillow.parquet')
equifax = \
pd.read_csv('data/Equifax_Consumer_Expenditure_Shelter_Utilities_Detail_Comparisons.csv')

Again, let's inspect the head to see how the data is constructed. 

In [14]:
equifax.head()

,State,County,Year,Total Households,Average Household Income,Median Household Income,Total Shelter,Mortgage Interest,Mortgage Interest,Mortgage Interest.1,...,Property Management Fees,Housing For Someone At School,Lodging Away From Home,Electricity,Fuel Oil,Natural Gas,Septic Tank Cleaning,Trash And Garbage Collection,Water And Sewage,Cellular Phone Service
0,Alabama,Autauga,2022,"21,948","$76,021","$60,199","$9,749.22","$2,501.76","$2,435.16",$66.60,...,$99.91,$48.40,$169.74,"$1,799.45",$31.12,$254.27,$6.61,$199.30,$531.61,"$1,310.46"
1,Alabama,Baldwin,2022,"94,231","$84,952","$58,644","$10,313.66","$2,554.75","$2,479.48",$75.27,...,$117.66,$51.77,$188.50,"$1,850.59",$32.77,$257.41,$7.21,$203.25,$538.04,"$1,312.10"
2,Alabama,Barbour,2022,"9,485","$53,115","$38,535","$7,660.23","$1,782.69","$1,737.90",$44.79,...,$77.36,$28.61,$123.81,"$1,560.99",$24.14,$212.78,$5.98,$178.79,$451.92,"$1,099.28"
3,Alabama,Bibb,2022,"7,878","$60,331","$50,337","$9,163.69","$2,265.62","$2,204.02",$61.60,...,$94.39,$43.88,$155.50,"$1,751.56",$29.78,$240.92,$6.50,$195.68,$510.20,"$1,249.50"
4,Alabama,Blount,2022,"22,261","$69,606","$55,747","$9,517.74","$2,397.33","$2,332.70",$64.63,...,$100.30,$46.65,$165.43,"$1,787.76",$31.18,$249.19,$6.75,$198.24,$526.79,"$1,286.02"


##### Transforming the Data

The Equifax data came in fairly ordered, so all that's needed are string manipulations to fix county names, summing each individual utility cost to form a "Total Utility Costs" metric and dropping uninformative columns, e.g. *Mortgage Interest*, for the purposes of our analysis. We say "uninformative" in this case, because while *Mortgage Interest* is part of overall shelter costs, we believe this metric is more subject to vary due to actions taken by the Fed and/or the US government rather than pure market forces. Mortgage rates around the pandemic timeframe (2020-2022) didn't change all that much so we assume that leaving them out of the affordability index doesn't change the overall results. We understand that this is a potential limitation and assumption in our analysis, and we highlight that in our report as a proposed future consideration/next steps. 

In [15]:
# list unique zillow counties to filter Equifax data on
counties_to_include = list(zillow['County'].unique())
# drop extra mortgage interest & property tax cols as well as avg household income
equifax = equifax.drop(equifax.columns[[4, 8, 9, 11, 12]], axis=1)

# fix spelling State names
equifax.State.replace("Conneticut","Connecticut", inplace=True)
equifax.State.replace("Deleware","Delaware", inplace=True)

equifax.loc[(equifax.State == 'Louisiana') & (equifax.County == 'La Salle'), 'County'] = "LaSalle"
equifax.County = equifax.County.str.replace("De Witt", "DeWitt")
equifax.County = equifax.County.str.replace("De Soto", "DeSoto")

# manipulate string columns in order to add all utilities to create total utility costs
equifax['Median Household Income'] = \
equifax['Median Household Income'].str.replace("$", "", regex=False).str.replace(",", "", regex=False).astype("float")

equifax['Total Households '] = \
equifax['Total Households '].str.replace("$", "", regex=False).str.replace(",", "", regex=False).astype("float")

equifax['Total Shelter'] = \
equifax['Total Shelter'].str.replace("$", "", regex=False).str.replace(",", "", regex=False).astype("float")

equifax.iloc[:, 6:] = \
equifax.iloc[:, 6:].applymap(lambda x: x.replace("$", "").replace(",", "")).astype('float')

# sum all utility costs to form a Total Utility Costs field
equifax['Total Utility Costs'] = equifax.iloc[:, 6:].sum(axis=1)
equifax = equifax.drop(columns=['Total Shelter'])
equifax['Total Utility Costs'] = equifax['Total Utility Costs'].apply(lambda x: round(x, 2))

# rows 552 & 2410 contain nan values for the year 2022, so let's drop them
equifax = equifax.drop([552, 2410])

# fix county and state strings
equifax['County'] = equifax['County'].str.strip()
equifax['State'] = equifax['State'].str.strip()

# remove the word 'County', 'Parish', 'Borough' from the end of County field name
equifax['County'] = equifax['County'].apply(lambda x: x.replace("County", "")).str.strip()
equifax['County'] = equifax['County'].apply(lambda x: x.replace("Parish", "")).str.strip()
equifax['County'] = equifax['County'].apply(lambda x: x.replace("Borough", "")).str.strip()

# filter equifax data to only include counties housed in the zillow dataset
equifax = equifax[equifax['County'].isin(counties_to_include)]


Let's inspect our 2022 for any nan or outliers. Since 2027 data only provides projections of utilities costs and not data on household income, we have a high percentage of nans as shown below. But we'll leave those nans, since we will use our 2027 utilties data further down to backwards interpolate 2022-2017 utilities data.

In [16]:
# make sure 2022 data is clean. 
display('2022 nans')
display(equifax.loc[equifax['Year'] == 2022].isna().sum() / equifax.shape[0])
display('2027 nans')
display(equifax.loc[equifax['Year'] == 2027].isna().sum() / equifax.shape[0])


'2022 nans'

State                            0.0
County                           0.0
Year                             0.0
Total Households                 0.0
Median Household Income          0.0
Mortgage Interest                0.0
Property Taxes                   0.0
Rental Costs                     0.0
Property Management Fees         0.0
Housing For Someone At School    0.0
Lodging Away From Home           0.0
Electricity                      0.0
Fuel Oil                         0.0
Natural Gas                      0.0
Septic Tank Cleaning             0.0
Trash And Garbage Collection     0.0
Water And Sewage                 0.0
Cellular Phone Service           0.0
Total Utility Costs              0.0
dtype: float64

'2027 nans'

State                            0.000000
County                           0.000000
Year                             0.000000
Total Households                 0.499914
Median Household Income          0.499914
Mortgage Interest                0.000000
Property Taxes                   0.000000
Rental Costs                     0.000000
Property Management Fees         0.000000
Housing For Someone At School    0.000000
Lodging Away From Home           0.000000
Electricity                      0.000000
Fuel Oil                         0.000000
Natural Gas                      0.000000
Septic Tank Cleaning             0.000000
Trash And Garbage Collection     0.000000
Water And Sewage                 0.000000
Cellular Phone Service           0.000000
Total Utility Costs              0.000000
dtype: float64

In [17]:
equifax.loc[equifax['Year'] == 2022, ['Median Household Income', 'Total Utility Costs']].describe()

,Median Household Income,Total Utility Costs
count,2917.000000,2917.000000
mean,57091.487144,9218.027384
std,14437.125627,1572.796275
min,25932.000000,5557.020000
25%,47317.000000,8061.370000
50%,54715.000000,9187.150000
75%,63450.000000,10022.990000
max,140868.000000,15600.980000


In [18]:
# write out equifax data to parquet to maintain dtypes and for faster read/write
equifax_filepath = 'data/equifax.parquet'
equifax.to_parquet(equifax_filepath)

#### ACS

Finally, onto the ACS data. ACS (American Community Survey) provides census data such as household income by demographic and age on a county level. The income field used formed the last main part of our affordability index metric, the main metric measuring housing affordability in our analysis. 

##### Setup file list for concatentation and build helper functions to concat and clean data

In [19]:
year_files = \
['ACSST5Y2016.csv', 'ACSST5Y2017.csv', 'ACSST5Y2018.csv', 'ACSST5Y2019.csv', 'ACSST5Y2020.csv','ACSST5Y2021.csv']

Let's build up our ACS data. We've constructed our file list above, which contains all the individual CSV files representing each year we're measuring. Below we'll create a helper function to concat all the ACS year files (each year from 2016 to 2021) into one dataset

In [20]:
def append_acs_data(year_files):
    '''
        Loops through each CSV year file and reads it into a dataframe
        Concatenates each dataframe into a final df

        year_files: list containing each CSV year file
        returns: final, concatenated dataframe
    '''
    
    types = defaultdict(str, income='float')
    # loop through each year in the year file list and read CSV into a df
    for year in year_files:
        if '2016' in year:
            acs_5_year_2016 = pd.read_csv('data/' + year, dtype=types, keep_default_na=False)
            acs_5_year_2016['Year'] = 2016
        elif '2017' in year:
            acs_5_year_2017 = pd.read_csv('data/' + year, dtype=types, keep_default_na=False)
            acs_5_year_2017['Year'] = 2017
        elif '2018' in year:
            acs_5_year_2018 = pd.read_csv('data/' + year, dtype=types, keep_default_na=False)
            acs_5_year_2018['Year'] = 2018
        elif '2019' in year:
            acs_5_year_2019 = pd.read_csv('data/' + year, dtype=types, keep_default_na=False)
            acs_5_year_2019['Year'] = 2019
        elif '2020' in year:
            acs_5_year_2020 = pd.read_csv('data/' + year, dtype=types, keep_default_na=False)
            acs_5_year_2020['Year'] = 2020
        elif '2021' in year:
            acs_5_year_2021 = pd.read_csv('data/' + year, dtype=types, keep_default_na=False)
            acs_5_year_2021['Year'] = 2021
        else:
            return print("Something went wrong")
    # concat all year df's into one
    final_df = pd.concat([acs_5_year_2016, acs_5_year_2017, acs_5_year_2018, acs_5_year_2019, acs_5_year_2020, acs_5_year_2021])     
    # reset index so each index ranges from 0 to len of df
    final_df = final_df.reset_index()
    return final_df

Now, let's create a function to clean the concatenated dataset by only keeping Median Income, State & County. We'll melt our data from wide to long as each column represented a different county (not helpful when trying to merge by State/County/Year) and clean our County and State strings.

In [21]:
def clean_acs_data(df):

    '''
        Filters concatenated df by only selecting Median Income, Year, 
        and Label (Grouping) column containing Households
        
        Cleans State and County strings 

        df: final concatenated df
        returns: cleaned, final concatenated df
    '''
    
    df['Label (Grouping)'] = df['Label (Grouping)'].str.strip()

    cols = [col for col in df.columns if '!!Median income (dollars)!!Estimate' in col or 'Label (Grouping)' in col or 'Year' in col]
    df = df[cols]

    df_median_household_income = df[df['Label (Grouping)'] == 'Households']

    df_median_household_income = \
    df_median_household_income.melt(id_vars=['Label (Grouping)', 'Year'], var_name='County', value_name='Median Household Income')

    df_median_household_income['County'] = \
    df_median_household_income['County'].str.replace("!!Median income (dollars)!!Estimate", "", regex=False)

    df_median_household_income['State'] = \
    df_median_household_income['County'].str.split(",").apply(lambda x: x[1].strip())

    df_median_household_income['County'] = \
    df_median_household_income['County'].str.split(",").apply(lambda x: x[0].strip()).str.replace("County", "", regex=False).str.strip()

    df_median_household_income['Median Household Income'] = \
    df_median_household_income['Median Household Income'].str.replace(",", "").str.extract("([0-9]+)").astype('float')

    return df_median_household_income

Below we'll handle the fixing of specific County names such as getting rid of  'Borough' and 'Parish' from Alaska and Louisiana respectively.

In [22]:
acs = clean_acs_data(append_acs_data(year_files))

acs['County'] = \
acs['County'].mask(acs['State'] == 'Alaska', acs['County'].str.replace('Borough', '')).str.strip()

acs['County'] = \
acs['County'].mask(acs['State'] == 'Alaska', acs['County'].str.replace('Municipality', '')).str.strip()

acs['County'] = \
acs['County'].mask(acs['State'] == 'Louisiana', acs['County'].str.replace('Parish', '')).str.strip()

acs['County'] = acs['County'].str.replace("De Witt", "DeWitt")
acs['County'] = acs['County'].str.replace("De Soto", "DeSoto")

# filter our final ACS dataset to only include counties within the Zillow dataset
acs = acs[acs['County'].isin(counties_to_include)]

# drop the label grouping field as the Median Income field contains the household income value
acs = acs.drop(columns=['Label (Grouping)'])


Let's inspect our ACS data for any nans or outliers

In [23]:
display('ACS nans')
display(acs[acs.isna().any(axis=1)].shape[0] / acs.shape[0])
display(acs.describe())

'ACS nans'

0.00011498217776244682

,Year,Median Household Income
count,17394.000000,17392.000000
mean,2018.500000,52655.691927
std,1.707874,14239.890162
min,2016.000000,18972.000000
25%,2017.000000,43209.750000
50%,2018.500000,50825.500000
75%,2020.000000,59196.250000
max,2021.000000,156821.000000


The nan percentage is very small, so let's drop nan's here. We'll also write our file out to parquet.

In [24]:
acs = acs.dropna()

acs_filepath = 'data/acs_5_year.parquet'
acs.to_parquet(acs_filepath)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=295bb911-c56a-4f18-9d98-f03c8b82d5f2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>